In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from PIL import Image

import utils as U

diode_path = Path("/home/gustavo/workstation/depth_estimation/data/datasets_quali/DIODE/")
diode_indoor = diode_path / "val" / "indoors"
diode_outdoor = diode_path / "val" / "outdoor"
diode_preds = Path("/home/gustavo/workstation/depth_estimation/data/outputs/DIODE-Anythingv2/npy/")

In [10]:
def eval_diode(gt_path, mask_path, pred_path):

    filenames_depth = U.get_sorted_files(gt_path, "_depth.npy")
    filenames_mask = U.get_sorted_files(mask_path, "_depth_mask.npy")
    filenames_preds = U.get_sorted_files(pred_path, ".npy")

    abs_rel_total = 0
    delta_total = 0

    num_imgs = len(filenames_depth)

    data = []

    for index in range(num_imgs):

        pred_diode = np.load(filenames_preds[index])
        groundtruth = np.squeeze(np.load(filenames_depth[index]))
        H,W = groundtruth.shape

        mask = np.load(filenames_mask[index]).reshape((H, W))
        mask_pred = np.ones_like(pred_diode)
        mask_pred[pred_diode < 1e-1] = 0
        mask_full = mask * mask_pred

        gt_depth_masked = np.zeros_like(groundtruth)
        gt_depth_masked[mask_full == 1] = groundtruth[mask_full == 1]

        gt_disp_masked = np.zeros_like(groundtruth)
        gt_disp_masked[mask_full == 1] = 1.0 / groundtruth[mask_full == 1]

        x_0, x_1 = U.compute_scale_and_shift(pred_diode, gt_disp_masked, mask_full)

        prediction_aligned = x_0 * pred_diode + x_1
        prediction_aligned = np.squeeze(prediction_aligned)*mask_full

        depth_aligned_masked = np.zeros_like(pred_diode)
        depth_aligned_masked[mask_full == 1] = 1 / prediction_aligned[mask_full == 1]

        abs_rel = U.abs_rel_error_mask(gt_depth_masked,depth_aligned_masked , mask_full)
        delta = U.calculate_delta(gt_depth_masked, depth_aligned_masked , mask_full, threshold=1.25)

        abs_rel_total = abs_rel_total + abs_rel
        delta_total = delta_total + delta

        row = {"fname": index, "absrel": abs_rel, "delta": delta}
        data.append(row)
    
    return abs_rel_total / num_imgs, delta_total/num_imgs, pd.DataFrame(data)

In [11]:
absrel, delta, df = eval_diode(diode_outdoor, diode_outdoor, diode_preds)

/tmp/ipykernel_24138/2886252211.py:37: RuntimeWarning: divide by zero encountered in divide
  depth_aligned_masked[mask_full == 1] = 1 / prediction_aligned[mask_full == 1]
/home/gustavo/workstation/depth_estimation/codes/depth-metrics-pavic/utils.py:262: RuntimeWarning: invalid value encountered in divide
  rel_diff[mask==1] = diff[mask==1] / (img2[mask==1])


In [12]:
absrel

np.float32(nan)

In [13]:
delta

np.float64(40.264071425429165)

In [14]:
df.head()

,fname,absrel,delta
0,0,0.828009,36.875188
1,1,0.694217,17.988570
2,2,0.041233,94.033256
3,3,1.144395,13.880389
4,4,0.084752,92.622166


In [15]:
df.tail()

,fname,absrel,delta
441,441,0.313667,17.241360
442,442,0.124660,16.082664
443,443,0.286338,26.604450
444,444,0.145066,32.865401
445,445,0.181382,34.365379


In [16]:
df.to_csv("diode_results.csv", index=False)